# Pre-procesamiento y consolidación de las fuentes de datos:
### Carga de librearias necesarias


In [1]:
import pandas as pd
import numpy as np

## Carga de archivos
Inicialmente  se carga la hoja de CARATULAS para obtener los NIT de las empresas como clave primaria para la consolidación, tambien se obtiene de esta hoja la clasificación CIIU de las empresas, su nombre como razón social y el estado, posterior mente se filtra por estado ACTIVA luego se **seleccionan solo las empresas que tienen REGISTROS EN TODOS LOS PERIODOS DE ANÁLISIS**

In [2]:
cara2015 = pd.read_excel('data/Plenas - Individuales15.xlsx', sheet_name='Caratula')
cara2017 = pd.read_excel('data/Plenas - Individuales17.xlsx', sheet_name='Caratula')
cara2019 = pd.read_excel('data/Plenas - Individuales19.xlsx', sheet_name='Carátula')

columnas = ['Nit', 'Clasificación Industrial Internacional Uniforme Versión 4 A.C',
                     'Razón social de la sociedad', 'Estado actual']

cara2015 = cara2015[['Nit',
                     'Razón social de la sociedad', 'Estado actual']]
cara2015 = cara2015[cara2015['Estado actual']=='ACTIVA']

cara2017 = cara2017[columnas]
cara2019 = cara2019[columnas]
# PARA ASEGURARNOS QUE LA EMPRESA TENGA REGISTROS PARA TODOS LOS PERIODOS
cara2017 = cara2017[cara2017.Nit.isin(cara2015.Nit.unique())]
cara2019 = cara2019[cara2019.Nit.isin(cara2017.Nit.unique())]
sector = pd.concat([cara2015, cara2017, cara2019])
sector.Nit = sector.Nit.astype(int)
sector.tail()

,Nit,Razón social de la sociedad,Estado actual,Clasificación Industrial Internacional Uniforme Versión 4 A.C
2504,900189254,COMERCIAL DE ENERGETICOS S.A SUCURSAL COLOMBIA...,ACTIVA,C2712 - Fabricación de aparatos de distribució...
2508,830052583,CANALES ANDRADE Y CIA SAS,ACTIVA,L6810 - Actividades inmobiliarias realizadas c...
2513,900173460,MADEMECO S.A.,ACTIVA,C1630 - Fabricación de partes y piezas de made...
2520,830117106,Desca Colombia SA,ACTIVA,"G4652 - Comercio al por mayor de equipo, parte..."
2529,860001584,MECANELECTRO SAS,ACTIVA,G4719 - Comercio al por menor en establecimien...


## Selección inicial del sector manufacturero según la clasificación CIIU

In [3]:
sector = sector[sector['Estado actual']=='ACTIVA']
sector = sector.drop_duplicates(['Nit', 'Clasificación Industrial Internacional Uniforme Versión 4 A.C'])
sector.dropna(inplace=True)
sector.Nit = sector.Nit.astype(int)
sector['codigo_sector'] = sector['Clasificación Industrial Internacional Uniforme Versión 4 A.C'].apply(lambda x: x.split('-')[0])
sector['nombre_sector'] = sector['Clasificación Industrial Internacional Uniforme Versión 4 A.C'].apply(lambda x: x.split('-')[1])
sector['filtro'] = sector.codigo_sector.apply(lambda x: True if x.startswith('C') else False)
sector = sector[sector.filtro]
sector.drop(['Clasificación Industrial Internacional Uniforme Versión 4 A.C', 'filtro', 'Estado actual'], axis=1, inplace=True)
sector = sector.rename(columns={'Razón social de la sociedad': 'nombre_empresa'})

# Consolidación de las hojas de balance y estados de resultados

#### CONSOLIDADO DE ESTADO DE RESULTADOS

In [4]:
plena2015 = pd.read_excel('data/Plenas - Individuales15.xlsx', sheet_name='ERI')
plena2017 = pd.read_excel('data/Plenas - Individuales17.xlsx', sheet_name='Estado de Resultado Integral')
plena2019 = pd.read_excel('data/Plenas - Individuales19.xlsx', sheet_name='ERI')

var = ['Nit','Periodo', 'Ganancia (pérdida), antes de impuestos', 'Costo de ventas', 'Ingresos de actividades ordinarias']

plena2015 = plena2015[var]
plena2017 = plena2017[var]
plena2019 = plena2019[var]
plena2015.Periodo = plena2015.Periodo.apply(lambda x : x.split('-')[0]).astype(int)
plena2017.Periodo = plena2017.Periodo.apply(lambda x : x.split('-')[0]).astype(int)
plena2019.Periodo = plena2019.Periodo.replace('Periodo Actual', 2019)
plena2019.Periodo = plena2019.Periodo.replace('Periodo Anterior', 2018)

data2 = pd.concat([plena2015, plena2017,  plena2019], axis=0, join='inner')
data2 = pd.merge(sector, data2, how='inner', on='Nit')
data2 = data2.drop_duplicates(['Nit', 'Periodo'])
data2 = data2.sort_values('Nit')
data2.head()

,Nit,nombre_empresa,codigo_sector,nombre_sector,Periodo,"Ganancia (pérdida), antes de impuestos",Costo de ventas,Ingresos de actividades ordinarias
0,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2015,1299641.0,31772654.0,39523443.0
1,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2014,931580.0,27295782.0,33605046.0
2,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2016,2056275.0,34861742.0,45830884.0
3,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2017,2804236.0,31607000.0,44780403.0
4,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2019,2179074.0,37681011.0,50538707.0


#### CONSOLIDACIÓN BALANCE GENERAL

In [5]:
plena2015 = pd.read_excel('data/Plenas - Individuales15.xlsx', sheet_name='ESF')
plena2017 = pd.read_excel('data/Plenas - Individuales17.xlsx', sheet_name='Estado de Situacion Financiera')
plena2019 = pd.read_excel('data/Plenas - Individuales19.xlsx', sheet_name='ESF')

var = ['Nit','Periodo', 'Total pasivos', 'Total de activos', 'Patrimonio total']


MAPPER= {
    'Pasivos':'Total pasivos',
    'Activos': 'Total de activos',
    'Patrimonio': 'Patrimonio total',
}
# RENOMBRADO DE COLUMNAS
plena2017 = plena2017.rename(columns=MAPPER)
plena2015 = plena2015[var]
plena2017 = plena2017[var]
plena2019 = plena2019[var]
plena2015.Periodo = plena2015.Periodo.apply(lambda x : x.split('-')[0]).astype(int)
plena2017.Periodo = plena2017.Periodo.apply(lambda x : x.split('-')[0]).astype(int)
plena2019.Periodo = plena2019.Periodo.replace('Periodo Actual', 2019)
plena2019.Periodo = plena2019.Periodo.replace('Periodo Anterior', 2018)
data = pd.concat([plena2015, plena2017,  plena2019], axis=0, join='inner')
data.head()

,Nit,Periodo,Total pasivos,Total de activos,Patrimonio total
0,900191874,2015,355261.0,623936.0,268675.0
1,900191874,2014,3079207.0,3634713.0,555506.0
2,900191874,2014,3619356.0,4241322.0,621966.0
3,900686320,2015,31317613.0,31372123.0,54510.0
4,900686320,2014,27237465.0,27228876.0,-8589.0


## Concatenación de balance y estado de resultados

In [6]:
data = pd.merge(data2, data, how='left', on=['Nit', 'Periodo'])
data = data.drop_duplicates(['Nit', 'Periodo'])
data.Periodo = data.Periodo.astype(int)
data = data[data.Periodo != 2014]
data.sort_values(['Nit', 'Periodo'], inplace=True)
data.head()

,Nit,nombre_empresa,codigo_sector,nombre_sector,Periodo,"Ganancia (pérdida), antes de impuestos",Costo de ventas,Ingresos de actividades ordinarias,Total pasivos,Total de activos,Patrimonio total
0,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2015,1299641.0,31772654.0,39523443.0,30497365.0,39392241.0,8894876.0
3,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2016,2056275.0,34861742.0,45830884.0,30231968.0,40278349.0,10046381.0
4,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2017,2804236.0,31607000.0,44780403.0,38682623.0,49114658.0,10432035.0
6,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2018,1923145.0,30396501.0,42692687.0,40402696.0,51176948.0,10774252.0
5,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2019,2179074.0,37681011.0,50538707.0,38659422.0,50246639.0,11587217.0


## Obtener valores reales descontado la inflación
Para la realización del ejercicio las variables son descontados según la inflación anual y teniendo en cuenta como año base el 2019.


In [7]:


def get_inflacion(value,  anio):
    inflación_año = {
        2015: (1.0459505653451563, 1.0775696, 1.0410576, 1.03589959),
        2016: (1.0775696, 1.0410576, 1.03589959),
        2017: (1.0410576, 1.03589959),
        2018: (1.0358995,),
        2019: (1,)
    }
    for item in inflación_año[anio]:
        value = value/item
    return value

def get_value_real(df, column_input, column_output):
    df[column_output] = ''
    for item in [2015, 2016, 2017, 2018,2019]:
        df[column_output] =  df.apply(lambda row: get_inflacion(row[column_input], item)
        if row['Periodo'] == item else row[column_output],
                                      axis=1)
    return df


data = get_value_real(data, 'Ganancia (pérdida), antes de impuestos', 'ganancia_antes_impuesto_real')
data = get_value_real(data, 'Total de activos', 'total_activos_real')
data = get_value_real(data, 'Total pasivos', 'total_pasivo_real')
data = get_value_real(data, 'Patrimonio total', 'patrimonio_total_real')
data = get_value_real(data, 'Ingresos de actividades ordinarias', 'ingresos_ordinarios_reales')
data = get_value_real(data, 'Costo de ventas', 'costo_venta_real')
data.head()

,Nit,nombre_empresa,codigo_sector,nombre_sector,Periodo,"Ganancia (pérdida), antes de impuestos",Costo de ventas,Ingresos de actividades ordinarias,Total pasivos,Total de activos,Patrimonio total,ganancia_antes_impuesto_real,total_activos_real,total_pasivo_real,patrimonio_total_real,ingresos_ordinarios_reales,costo_venta_real
0,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2015,1299641.0,31772654.0,39523443.0,30497365.0,39392241.0,8894876.0,1.069238e+06,3.240871e+07,2.509074e+07,7.317976e+06,3.251665e+07,2.613994e+07
3,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2016,2056275.0,34861742.0,45830884.0,30231968.0,40278349.0,10046381.0,1.769471e+06,3.466043e+07,2.601529e+07,8.645137e+06,3.943851e+07,2.999931e+07
4,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2017,2804236.0,31607000.0,44780403.0,38682623.0,49114658.0,10432035.0,2.600292e+06,4.554269e+07,3.586935e+07,9.673344e+06,4.152366e+07,2.930832e+07
6,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2018,1923145.0,30396501.0,42692687.0,40402696.0,51176948.0,10774252.0,1.856498e+06,4.940339e+07,3.900252e+07,1.040087e+07,4.121316e+07,2.934310e+07
5,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2019,2179074.0,37681011.0,50538707.0,38659422.0,50246639.0,11587217.0,2.179074e+06,5.024664e+07,3.865942e+07,1.158722e+07,5.053871e+07,3.768101e+07


### Cálculo de la propoción costo ingreso

In [8]:
data['cv/io'] = data['costo_venta_real']/data['ingresos_ordinarios_reales']

## Clasificación y filtro de las empresa según su tamaño
Para disminuir un poco la viarabilidad entre empresas se calsaificarón las empresas seǵun su tamaño de acuerdo a los ingresos ordinarios de estas dado por el Decreto 957 de 2019 se clasificaron  estas empresas de alimentos en micro, pequeña, mediana y grande, seleccionando las medianas y grandes.
**https://www.mincit.gov.co/prensa/noticias/industria/gobierno-expide-nueva-clasificacion-de-empresas-a**


In [9]:
data['tipo_empresa'] = np.where(data['Ingresos de actividades ordinarias'] <= 811000, 'micro',
                                 np.where(data['Ingresos de actividades ordinarias'] <= 7027000, 'pequeña',
                                     np.where(data['Ingresos de actividades ordinarias'] <= 59527000, 'mediana','grande')))
# Filtro: solo mediana y grande
data =data[data['tipo_empresa'].isin(['mediana','grande'])]
data.head()

,Nit,nombre_empresa,codigo_sector,nombre_sector,Periodo,"Ganancia (pérdida), antes de impuestos",Costo de ventas,Ingresos de actividades ordinarias,Total pasivos,Total de activos,Patrimonio total,ganancia_antes_impuesto_real,total_activos_real,total_pasivo_real,patrimonio_total_real,ingresos_ordinarios_reales,costo_venta_real,cv/io,tipo_empresa
0,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2015,1299641.0,31772654.0,39523443.0,30497365.0,39392241.0,8894876.0,1.069238e+06,3.240871e+07,2.509074e+07,7.317976e+06,3.251665e+07,2.613994e+07,0.803894,mediana
3,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2016,2056275.0,34861742.0,45830884.0,30231968.0,40278349.0,10046381.0,1.769471e+06,3.466043e+07,2.601529e+07,8.645137e+06,3.943851e+07,2.999931e+07,0.760660,mediana
4,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2017,2804236.0,31607000.0,44780403.0,38682623.0,49114658.0,10432035.0,2.600292e+06,4.554269e+07,3.586935e+07,9.673344e+06,4.152366e+07,2.930832e+07,0.705822,mediana
6,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2018,1923145.0,30396501.0,42692687.0,40402696.0,51176948.0,10774252.0,1.856498e+06,4.940339e+07,3.900252e+07,1.040087e+07,4.121316e+07,2.934310e+07,0.711984,mediana
5,800003267,integrafic de occidente SA,C1811,Actividades de impresión,2019,2179074.0,37681011.0,50538707.0,38659422.0,50246639.0,11587217.0,2.179074e+06,5.024664e+07,3.865942e+07,1.158722e+07,5.053871e+07,3.768101e+07,0.745587,mediana


## Selección de empresas manufactureras pertencientes al sector de alimentos
Luego de los filtros aplicados se procedio a seleccionar solo las empresas manufactureras que pertenecen la sector de alimentos,para finalmente obtener un **total de 42 empresas**

In [ ]:
data2 = pd.read_excel('data/consolidado_ESF_ERI_alimentos.xlsx', sheet_name='alimentos')
data = data[data.Nit.isin(data2.Nit.unique())]
data = data[data.Nit != 900285758]
len(data.Nit.unique())

## Indicadores macroeconomicos y commoditys
En este punto al consolidarón de las variables de las empresas con las variables macroeconomicas y de commoditys sgún el periodo correspondiente, tambien se llevan a valores reales estas variables se llevan a pesos colombianos los commoditys

In [ ]:
##### Archvio con indicadores macro
macro = pd.read_excel('data/insumo_var_macro_hist.xlsx', sheet_name='resumen')
data.sort_values(['Nit', 'Periodo'], inplace=True)
data = pd.merge(data, macro, how='left', on='Periodo')

data = get_value_real(data, 'Exportaciones ', 'exportaciones_real')
data = get_value_real(data, 'Importaciones', 'importaciones_real')
data['balanza_comercial'] = data.exportaciones_real - data.importaciones_real
data['balanza/pib'] = data['balanza_comercial']/data.PIB_real
data = get_value_real(data, 'TRM', 'trm_real')

for item in ['BRENT', 'WTI', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar']:
    data[item] = data[item]*data.trm_real
data.head()
data.describe()

## Cálculo de indicadores
En este apartado se calcularon los indicadores que se pensarón utilizar como variables respuesta en la parte de modelamiento, aunque no todos se utilizarón para el análisis, estos fueron:
- ROA
- ENDEUDAMIENTO
- VARIACIÓN DE LA RELACIÓN COSTO INGRESO
- VARIACIÓN DE LOS COSTOS

In [ ]:
# ROA
#Rentabilidad Retorno sobre los Activos (ROA) (ERI- Ganancia (pérdida), antes de impuestos / ESF-Total de activos) *100
data['ROA'] = data['ganancia_antes_impuesto_real'] / data['total_activos_real']

# ENDEUDAMIENTO :
# Apalancamiento Financiero,  ESF- Total pasivos / ESF- Patrimonio total
data['apalancamiento'] = data['total_pasivo_real']/data['patrimonio_total_real']

# VARIACIÓN DE LA RELACIÓN COSTO INGRESO
indicador = list()
for ind, value in data['cv/io'].iteritems():
    if data['Periodo'][ind] != 2015:
        indicador.append((data['cv/io'][ind] - data['cv/io'][ind-1])/data['cv/io'][ind-1])
    else:
        indicador.append(1)
data['var_costo_norma'] =  indicador

# VARIACIÓN DE LOS COSTOS
# ((ERI- Costo de Ventas año actual - ERI- Costo de Ventas año anterior) / ERI- Costo de Ventas) x 100
indicador = list()
for ind, value in data['costo_venta_real'].iteritems():
    if data['Periodo'][ind] != 2015:
        indicador.append((data['costo_venta_real'][ind] - data['costo_venta_real'][ind-1])/data['costo_venta_real'][ind-1])
    else:
        indicador.append(1)
data['var_costo'] =  indicador
data.to_csv('data/consolidado.csv', index=False, sep=';')
data = data[data.Periodo != 2015]
data.head()

## Selección de columnas para la etapa de modelado

In [ ]:
var = ['Nit', 'Periodo','PIB_real', 'PIB _var', 'Deflactor_del_PIB',
       'Consumo_real_(COP_miles_de_millones)', 'Consumo_real_(var._%_anual)',
       'Tasa_de_desempleo', 'Cuenta Corriente ', 'Exportaciones ',
       'var_Exportaciones', 'Importaciones', 'var_Importaciones', 'IPC',
       'Inflacion_var', 'Tasa_de_intervencion', 'DTF', 'IBR_overnight', 'TRM',
       'COLCAP', 'BRENT', 'WTI','var_salario', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar',
        'ROA', 'apalancamiento', 'var_costo_norma', 'var_costo']
data = data[var]
mapper = { 'Consumo_real_(COP_miles_de_millones)': 'consumo_real',
            'Consumo_real_(var._%_anual)':'Consumo_real_var',
            'PIB _var': 'PIB_var',
}
data1 = data
data1 = data1.rename(columns=mapper)
data1.columns = map(str.lower, data1.columns)
data1.to_csv('data/datos.csv', index=False, sep=';')
data1.head()

## Conjunto de datos con variables explicativas en t-1
Para ampliar las posibilidades de análisis se decidio crear otros conjuntos de datos en dondes las variables explicativas tendrían un retardo de un tiempo con el fin de anlizar estas relaciones, asi por ejemplo la varible respuestas  del periodo 2016 esta sociada a las variables explicativas macroeconomicas y de commoditys del perido 2015.

In [ ]:
periodo = data1.periodo

macro.Periodo = macro.Periodo + 1
data_t1 = data.drop(macro.columns, axis=1)

data_t1['Periodo'] = periodo
data_t1 = pd.merge(data_t1, macro, how='left', left_on='Periodo', right_on='Periodo')
data_t1 = get_value_real(data_t1, 'Exportaciones ', 'exportaciones_real')
data_t1 = get_value_real(data_t1, 'Importaciones', 'importaciones_real')
data_t1['balanza_comercial'] = data_t1.exportaciones_real - data_t1.importaciones_real
data_t1['balanza/pib'] = data_t1['balanza_comercial']/data_t1.PIB_real
data_t1 = get_value_real(data_t1, 'TRM', 'trm_real')

for item in ['BRENT', 'WTI', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar']:
    data_t1[item] = data_t1[item]*data_t1.trm_real
data_t1 = data_t1.rename(columns=mapper)
data_t1.columns = map(str.lower, data_t1.columns)


data_t1.to_csv('data/datos_t1.csv', index=False, sep=';')
data_t1.head()

## Conjunto de datos con variables explicativas en t-2
De igual forma que en el partado anterior, las varaibles respuestas tiene un retardo pero en este caso de dos periodos asi por ejemplo la varible respuestas del periodo 2016 esta sociada a las variables explicativas macroeconomicas y de commoditys del perido 2014.

In [ ]:
#datos macro t-2
macro.Periodo = macro.Periodo + 1
data_t2 = data.drop(macro.columns, axis=1)

data_t2['Periodo'] = periodo
data_t2 = pd.merge(data_t2, macro, how='left', on='Periodo')
data_t2 = get_value_real(data_t2, 'Exportaciones ', 'exportaciones_real')
data_t2 = get_value_real(data_t2, 'Importaciones', 'importaciones_real')
data_t2['balanza_comercial'] = data_t2.exportaciones_real - data_t2.importaciones_real
data_t2['balanza/pib'] = data_t2['balanza_comercial']/data_t2.PIB_real
data_t2 = get_value_real(data_t2, 'TRM', 'trm_real')

for item in ['BRENT', 'WTI', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar']:
    data_t2[item] = data_t2[item]*data_t2.trm_real
data_t2 = data_t2.rename(columns=mapper)
data_t2.columns = map(str.lower, data_t2.columns)
data_t2.to_csv('data/datos_t2.csv', index=False, sep=';')
data_t2.head()

In [10]:
#datos macro t-2
macro.Periodo = macro.Periodo + 1
data_t2 = data.drop(macro.columns, axis=1)

data_t2['Periodo'] = periodo
data_t2 = pd.merge(data_t2, macro, how='left', on='Periodo')
data_t2 = get_value_real(data_t2, 'Exportaciones ', 'exportaciones_real')
data_t2 = get_value_real(data_t2, 'Importaciones', 'importaciones_real')
data_t2['balanza_comercial'] = data_t2.exportaciones_real - data_t2.importaciones_real
data_t2['balanza/pib'] = data_t2['balanza_comercial']/data_t2.PIB_real
data_t2 = get_value_real(data_t2, 'TRM', 'trm_real')

for item in ['BRENT', 'WTI', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar']:
    data_t2[item] = data_t2[item]*data_t2.trm_real
data_t2 = data_t2.rename(columns=mapper)
data_t2.columns = map(str.lower, data_t2.columns)
data_t2.to_csv('data/datos_t2.csv', index=False, sep=';')
data_t2.head()

42

## Indicadores macroeconomicos y commoditys
En este punto al consolidarón de las variables de las empresas con las variables macroeconomicas y de commoditys sgún el periodo correspondiente, tambien se llevan a valores reales estas variables se llevan a pesos colombianos los commoditys

In [11]:
##### Archvio con indicadores macro
macro = pd.read_excel('data/insumo_var_macro_hist.xlsx', sheet_name='resumen')
data.sort_values(['Nit', 'Periodo'], inplace=True)
data = pd.merge(data, macro, how='left', on='Periodo')

data = get_value_real(data, 'Exportaciones ', 'exportaciones_real')
data = get_value_real(data, 'Importaciones', 'importaciones_real')
data['balanza_comercial'] = data.exportaciones_real - data.importaciones_real
data['balanza/pib'] = data['balanza_comercial']/data.PIB_real
data = get_value_real(data, 'TRM', 'trm_real')

for item in ['BRENT', 'WTI', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar']:
    data[item] = data[item]*data.trm_real
data.head()
data.describe()

,Nit,Periodo,"Ganancia (pérdida), antes de impuestos",Costo de ventas,Ingresos de actividades ordinarias,Total pasivos,Total de activos,Patrimonio total,ganancia_antes_impuesto_real,total_activos_real,...,Food,Oils & Meals,Grains,Other Food,sugar,exportaciones_real,importaciones_real,balanza_comercial,balanza/pib,trm_real
count,2.100000e+02,210.000000,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02,2.100000e+02,...,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,8.539258e+08,2017.000000,1.586976e+07,1.442276e+08,1.995219e+08,8.350069e+07,1.764863e+08,9.298565e+07,1.446083e+07,1.622549e+08,...,249633.752852,236176.935633,238919.433503,277035.483152,896.666128,8608.753059,11399.358540,-2790.605481,-0.013336,2750.221980
std,4.047433e+07,1.417593,4.880197e+07,1.727862e+08,2.528775e+08,1.042627e+08,2.266348e+08,1.450661e+08,4.288628e+07,2.096926e+08,...,26547.201845,21619.946132,31254.168866,33251.427229,153.568617,1306.689492,1245.270036,672.762615,0.003348,333.009762
min,8.000410e+08,2015.000000,-4.556303e+07,6.888773e+06,9.360072e+06,1.489517e+06,6.212884e+06,-3.224668e+07,-3.920801e+07,5.972920e+06,...,203875.351197,194419.167172,200650.731476,219234.605115,682.775454,6834.343347,9657.077972,-3710.474962,-0.018444,2255.251171
25%,8.110364e+08,2016.000000,7.559155e+05,3.601085e+07,4.712884e+07,1.920196e+07,4.871025e+07,2.501608e+07,6.219054e+05,4.606634e+07,...,250330.875947,238315.463745,225360.095449,277903.200628,772.679700,7408.067398,10681.188499,-3301.599600,-0.014974,2626.427956
50%,8.600279e+08,2017.000000,4.743633e+06,8.418626e+07,1.124646e+08,4.529993e+07,1.004330e+08,4.619081e+07,4.292225e+06,8.972810e+07,...,253567.594900,246191.331032,226441.102346,277918.139745,923.298984,8814.160419,11118.542360,-2822.734625,-0.013744,2736.045751
75%,8.917016e+08,2018.000000,1.510056e+07,1.714702e+08,2.443798e+08,1.021998e+08,2.188790e+08,8.408241e+07,1.475362e+07,2.018583e+08,...,253592.286062,246553.005774,248833.632827,287904.703083,992.759104,9874.056370,12364.327901,-2251.190140,-0.010549,2853.375942
max,9.005532e+08,2019.000000,5.223492e+08,8.760991e+08,1.381911e+09,5.576228e+08,1.432629e+09,1.003207e+09,4.494932e+08,1.232810e+09,...,286802.656153,255405.710442,293311.605415,322216.767189,1111.817398,10113.137761,13175.655970,-1867.028079,-0.008969,3280.009080


## Cálculo de indicadores
En este apartado se calcularon los indicadores que se pensarón utilizar como variables respuesta en la parte de modelamiento, aunque no todos se utilizarón para el análisis, estos fueron:
- ROA
- ENDEUDAMIENTO
- VARIACIÓN DE LA RELACIÓN COSTO INGRESO
- VARIACIÓN DE LOS COSTOS

In [12]:
# ROA
#Rentabilidad Retorno sobre los Activos (ROA) (ERI- Ganancia (pérdida), antes de impuestos / ESF-Total de activos) *100
data['ROA'] = data['ganancia_antes_impuesto_real'] / data['total_activos_real']

# ENDEUDAMIENTO :
# Apalancamiento Financiero,  ESF- Total pasivos / ESF- Patrimonio total
data['apalancamiento'] = data['total_pasivo_real']/data['patrimonio_total_real']

# VARIACIÓN DE LA RELACIÓN COSTO INGRESO
indicador = list()
for ind, value in data['cv/io'].iteritems():
    if data['Periodo'][ind] != 2015:
        indicador.append((data['cv/io'][ind] - data['cv/io'][ind-1])/data['cv/io'][ind-1])
    else:
        indicador.append(1)
data['var_costo_norma'] =  indicador

# VARIACIÓN DE LOS COSTOS
# ((ERI- Costo de Ventas año actual - ERI- Costo de Ventas año anterior) / ERI- Costo de Ventas) x 100
indicador = list()
for ind, value in data['costo_venta_real'].iteritems():
    if data['Periodo'][ind] != 2015:
        indicador.append((data['costo_venta_real'][ind] - data['costo_venta_real'][ind-1])/data['costo_venta_real'][ind-1])
    else:
        indicador.append(1)
data['var_costo'] =  indicador
data.to_csv('data/consolidado.csv', index=False, sep=';')
data = data[data.Periodo != 2015]
data.head()

,Nit,nombre_empresa,codigo_sector,nombre_sector,Periodo,"Ganancia (pérdida), antes de impuestos",Costo de ventas,Ingresos de actividades ordinarias,Total pasivos,Total de activos,...,sugar,exportaciones_real,importaciones_real,balanza_comercial,balanza/pib,trm_real,ROA,apalancamiento,var_costo_norma,var_costo
1,800041007,PROVIMI S.A.,C1090,Elaboración de alimentos preparados para anim...,2016,4005797.0,32371397.0,49221330.0,10344448.0,33072931.0,...,1111.817398,6834.343347,9657.077972,-2822.734625,-0.013744,2626.427956,0.121120,0.455131,0.125013,0.398836
2,800041007,PROVIMI S.A.,C1090,Elaboración de alimentos preparados para anim...,2017,10428511.0,51900823.0,76979359.0,19199817.0,48056605.0,...,992.759104,8814.160419,10681.188499,-1867.028079,-0.008969,2736.045751,0.217005,0.665348,0.025160,0.727659
3,800041007,PROVIMI S.A.,C1090,Elaboración de alimentos preparados para anim...,2018,8870421.0,60944040.0,88693950.0,14444062.0,48795569.0,...,772.679700,10113.137761,12364.327901,-2251.190140,-0.010549,2853.375942,0.181787,0.420478,0.019148,0.222452
4,800041007,PROVIMI S.A.,C1090,Elaboración de alimentos preparados para anim...,2019,8688054.0,76017570.0,107913346.0,35504930.0,75427224.0,...,923.298984,9874.056370,13175.655970,-3301.599600,-0.014974,3280.009080,0.115185,0.889351,0.025183,0.292113
6,800045228,DESCAFEINADORA COLOMBIANA S.A.S.,C1062,"Descafeinado, tostión y molienda del café",2016,6403632.0,34032537.0,46808899.0,10257248.0,65796013.0,...,1111.817398,6834.343347,9657.077972,-2822.734625,-0.013744,2626.427956,0.097326,0.184686,-0.058132,0.167658


## Selección de columnas para la etapa de modelado

In [13]:
var = ['Nit', 'Periodo','PIB_real', 'PIB _var', 'Deflactor_del_PIB',
       'Consumo_real_(COP_miles_de_millones)', 'Consumo_real_(var._%_anual)',
       'Tasa_de_desempleo', 'Cuenta Corriente ', 'Exportaciones ',
       'var_Exportaciones', 'Importaciones', 'var_Importaciones', 'IPC',
       'Inflacion_var', 'Tasa_de_intervencion', 'DTF', 'IBR_overnight', 'TRM',
       'COLCAP', 'BRENT', 'WTI','var_salario', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar',
        'ROA', 'apalancamiento', 'var_costo_norma', 'var_costo']
data = data[var]
mapper = { 'Consumo_real_(COP_miles_de_millones)': 'consumo_real',
            'Consumo_real_(var._%_anual)':'Consumo_real_var',
            'PIB _var': 'PIB_var',
}
data1 = data
data1 = data1.rename(columns=mapper)
data1.columns = map(str.lower, data1.columns)
data1.to_csv('data/datos.csv', index=False, sep=';')
data1.head()

,nit,periodo,pib_real,pib_var,deflactor_del_pib,consumo_real,consumo_real_var,tasa_de_desempleo,cuenta corriente,exportaciones,...,beverages,food,oils & meals,grains,other food,sugar,roa,apalancamiento,var_costo_norma,var_costo
1,800041007,2016,205372,0.020778,1.051299,139934.7500,0.015676,0.099991,-0.043057,7942.085245,...,254986.825670,250330.875947,246553.005774,225360.095449,277903.200628,1111.817398,0.121120,0.455131,0.125013,0.398836
2,800041007,2017,208164,0.013496,1.105498,142808.7500,0.020663,0.106159,-0.033325,9505.465078,...,233646.800270,253592.286062,246191.331032,226441.102346,287904.703083,992.759104,0.217005,0.665348,0.025160,0.727659
3,800041007,2018,213400,0.025008,1.155240,147104.5000,0.030118,0.107706,-0.039098,10476.194350,...,221721.152689,253567.594900,238315.463745,248833.632827,277918.139745,772.679700,0.181787,0.420478,0.019148,0.222452
4,800041007,2019,220490,0.033178,1.204665,153913.4328,0.046347,0.111948,-0.042644,9874.056370,...,250990.509388,286802.656153,255405.710442,293311.605415,322216.767189,923.298984,0.115185,0.889351,0.025183,0.292113
6,800045228,2016,205372,0.020778,1.051299,139934.7500,0.015676,0.099991,-0.043057,7942.085245,...,254986.825670,250330.875947,246553.005774,225360.095449,277903.200628,1111.817398,0.097326,0.184686,-0.058132,0.167658


## Conjunto de datos con variables explicativas en t-1
Para ampliar las posibilidades de análisis se decidio crear otros conjuntos de datos en dondes las variables explicativas tendrían un retardo de un tiempo con el fin de anlizar estas relaciones, asi por ejemplo la varible respuestas  del periodo 2016 esta sociada a las variables explicativas macroeconomicas y de commoditys del perido 2015.

In [14]:
periodo = data1.periodo

macro.Periodo = macro.Periodo + 1
data_t1 = data.drop(macro.columns, axis=1)

data_t1['Periodo'] = periodo
data_t1 = pd.merge(data_t1, macro, how='left', left_on='Periodo', right_on='Periodo')
data_t1 = get_value_real(data_t1, 'Exportaciones ', 'exportaciones_real')
data_t1 = get_value_real(data_t1, 'Importaciones', 'importaciones_real')
data_t1['balanza_comercial'] = data_t1.exportaciones_real - data_t1.importaciones_real
data_t1['balanza/pib'] = data_t1['balanza_comercial']/data_t1.PIB_real
data_t1 = get_value_real(data_t1, 'TRM', 'trm_real')

for item in ['BRENT', 'WTI', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar']:
    data_t1[item] = data_t1[item]*data_t1.trm_real
data_t1 = data_t1.rename(columns=mapper)
data_t1.columns = map(str.lower, data_t1.columns)


data_t1.to_csv('data/datos_t1.csv', index=False, sep=';')
data_t1.head()

,nit,roa,apalancamiento,var_costo_norma,var_costo,periodo,pib_real,pib_var,deflactor_del_pib,consumo_real,...,food,oils & meals,grains,other food,sugar,exportaciones_real,importaciones_real,balanza_comercial,balanza/pib,trm_real
0,800041007,0.121120,0.455131,0.125013,0.398836,2016,201173,0.029859,0.999717,137753.25,...,213243.538844,203352.837818,209870.746024,229308.559163,714.149372,7748.472283,11629.445667,-3880.973384,-0.019292,2358.881237
1,800041007,0.217005,0.665348,0.025160,0.727659,2017,205372,0.020778,1.051299,139934.75,...,269748.941862,265678.023811,242841.187909,299460.040739,1198.060629,7364.480627,10406.173647,-3041.693021,-0.014811,2830.158922
2,800041007,0.181787,0.420478,0.019148,0.222452,2018,208164,0.013496,1.105498,142808.75,...,264004.199643,256299.378493,235738.251031,299725.405260,1033.519500,9176.049489,11119.733430,-1943.683940,-0.009337,2848.381470
3,800041007,0.115185,0.889351,0.025183,0.292113,2019,213400,0.025008,1.155240,147104.50,...,262670.544773,246870.869736,257766.635829,287895.262003,800.418515,10476.194350,12808.201090,-2332.006740,-0.010928,2955.810712
4,800045228,0.097326,0.184686,-0.058132,0.167658,2016,201173,0.029859,0.999717,137753.25,...,213243.538844,203352.837818,209870.746024,229308.559163,714.149372,7748.472283,11629.445667,-3880.973384,-0.019292,2358.881237


## Conjunto de datos con variables explicativas en t-2
De igual forma que en el partado anterior, las varaibles respuestas tiene un retardo pero en este caso de dos periodos asi por ejemplo la varible respuestas del periodo 2016 esta sociada a las variables explicativas macroeconomicas y de commoditys del perido 2014.

In [15]:
#datos macro t-2
macro.Periodo = macro.Periodo + 1
data_t2 = data.drop(macro.columns, axis=1)

data_t2['Periodo'] = periodo
data_t2 = pd.merge(data_t2, macro, how='left', on='Periodo')
data_t2 = get_value_real(data_t2, 'Exportaciones ', 'exportaciones_real')
data_t2 = get_value_real(data_t2, 'Importaciones', 'importaciones_real')
data_t2['balanza_comercial'] = data_t2.exportaciones_real - data_t2.importaciones_real
data_t2['balanza/pib'] = data_t2['balanza_comercial']/data_t2.PIB_real
data_t2 = get_value_real(data_t2, 'TRM', 'trm_real')

for item in ['BRENT', 'WTI', 'Energy',	'Non-energy',
       'Agriculture',	'Raw Materials',	'Fertilizers',	'Beverages',
       'Food',	'Oils & Meals',	'Grains', 'Other Food',	'sugar']:
    data_t2[item] = data_t2[item]*data_t2.trm_real
data_t2 = data_t2.rename(columns=mapper)
data_t2.columns = map(str.lower, data_t2.columns)
data_t2.to_csv('data/datos_t2.csv', index=False, sep=';')
data_t2.head()

,nit,roa,apalancamiento,var_costo_norma,var_costo,periodo,pib_real,pib_var,deflactor_del_pib,consumo_real,...,food,oils & meals,grains,other food,sugar,exportaciones_real,importaciones_real,balanza_comercial,balanza/pib,trm_real
0,800041007,0.121120,0.455131,0.125013,0.398836,2016,195397,0.045566,0.976532,133621.00,...,168641.097397,169171.442873,165566.062760,170726.730154,596.494688,11801.368406,13774.574371,-1973.205964,-0.010098,1721.421131
1,800041007,0.217005,0.665348,0.025160,0.727659,2017,201173,0.029859,0.999717,137753.25,...,229784.754855,219126.836106,226150.335845,247095.932374,769.545653,8349.518178,12531.537115,-4182.018937,-0.020788,2541.858711
2,800041007,0.181787,0.420478,0.019148,0.222452,2018,205372,0.020778,1.051299,139934.75,...,280824.210415,276586.149871,252811.686230,311755.178393,1247.250231,7666.849192,10833.427104,-3166.577911,-0.015419,2946.358710
3,800041007,0.115185,0.889351,0.025183,0.292113,2019,208164,0.013496,1.105498,142808.75,...,273481.818408,265500.398031,244201.136374,310485.397446,1070.622333,9505.465078,11518.926300,-2013.461222,-0.009672,2950.636941
4,800045228,0.097326,0.184686,-0.058132,0.167658,2016,195397,0.045566,0.976532,133621.00,...,168641.097397,169171.442873,165566.062760,170726.730154,596.494688,11801.368406,13774.574371,-1973.205964,-0.010098,1721.421131
